> ⚠️ **작업 중 (Work in Progress)**: 이 노트북은 현재 개발 중입니다. 일부 코드가 불완전하거나 변경될 수 있습니다.

## 📋 목차

- [Control Plane 개요](#control-plane-개요)
- [Fleet Overview](#fleet-overview)
- [Assets 관리](#assets-관리)
- [Compliance 및 보안](#compliance-및-보안)
- [Quota 관리](#quota-관리)
- [Admin 기능](#admin-기능)

## 🎯 학습 목표

- Control Plane의 역할과 중요성 이해
- Fleet Overview를 통한 전체 시스템 모니터링
- Assets(에이전트, 모델, 도구) 관리 방법
- 컴플라이언스 및 보안 설정 구성
- Quota 및 Rate Limiting 관리
- 프로젝트 및 사용자 권한 관리

## ⏱️ 예상 소요 시간

약 10분

## Control Plane 개요

### Control Plane이란?

Control Plane은 Microsoft Foundry의 중앙 관리 센터로, 모든 AI 리소스를 통합 관리하고 모니터링합니다.

```
Control Plane = 모니터링 + 관리 + 보안 + 거버넌스
```

### 주요 기능 영역

```
┌─────────────────────────────────────────┐
│         Control Plane                   │
├─────────────────────────────────────────┤
│ Fleet Overview   │ 전체 시스템 대시보드   │
│ Assets          │ 리소스 관리           │
│ Compliance      │ 보안 및 정책          │
│ Quota           │ 할당량 및 제한        │
│ Admin           │ 프로젝트 및 권한      │
└─────────────────────────────────────────┘
```

### 왜 중요한가?

프로덕션 환경에서 다음을 보장합니다:
- 📊 **가시성**: 모든 리소스의 상태와 성능 파악
- 🛡️ **보안**: 취약점과 위협 조기 발견
- 💰 **비용 관리**: 리소스 사용량과 비용 최적화
- ⚖️ **컴플라이언스**: 규정 준수 상태 모니터링
- 🚨 **알림**: 문제 발생 시 즉각 대응

### Portal vs SDK

| 기능 | Portal | SDK (Python) |
|-----|--------|-------------|
| Fleet Overview | ✅ 대시보드 제공 | ⚠️ 제한적 조회 |
| Assets 조회 | ✅ 시각화 | ✅ 프로그래밍 방식 |
| Compliance 모니터링 | ✅ 실시간 대시보드 | ❌ 지원 안 함 |
| Quota 관리 | ✅ Portal에서 관리 | ❌ 조회만 가능 |
| Admin 설정 | ✅ RBAC 관리 | ❌ 지원 안 함 |

**💡 권장 사항:**
- **Portal**: 실시간 모니터링, 보안/컴플라이언스 확인, Quota 관리
- **SDK**: 자동화, CI/CD 파이프라인, 프로그래밍 방식 조회

## 환경 설정

In [ ]:
# 환경 변수 로드
import json
import os
import subprocess

# PATH 환경변수 설정 (Azure CLI를 찾을 수 있도록)
possible_paths = [
    "/opt/homebrew/bin",  # macOS (Apple Silicon)
    "/usr/local/bin",     # macOS (Intel) / Linux
    "/usr/bin",           # Linux / GitHub Codespaces
    "/home/linuxbrew/.linuxbrew/bin"  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
except:
    pass

paths_to_add = []
if az_path and az_path not in os.environ.get("PATH", ""):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get("PATH", ""):
            paths_to_add.append(path)

if paths_to_add:
    new_path = ":".join(paths_to_add) + ":" + os.environ.get("PATH", "")
    os.environ["PATH"] = new_path

# 이전 노트북에서 저장한 설정 파일 로드
config_file = ".foundry_config.json"
try:
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    # 환경 변수 설정
    FOUNDRY_NAME = config.get("FOUNDRY_NAME")
    RESOURCE_GROUP = config.get("RESOURCE_GROUP")
    LOCATION = config.get("LOCATION")
    TENANT_ID = config.get("TENANT_ID")
    PROJECT_NAME = config.get("PROJECT_NAME", "proj-default")
    PROJECT_ENDPOINT = config.get("FOUNDRY_ENDPOINT")
    
    # 환경 변수로도 설정 (다른 도구들이 사용할 수 있도록)
    os.environ["FOUNDRY_NAME"] = FOUNDRY_NAME
    os.environ["LOCATION"] = LOCATION
    os.environ["RESOURCE_GROUP"] = RESOURCE_GROUP
    os.environ["AZURE_SUBSCRIPTION_ID"] = config.get("AZURE_SUBSCRIPTION_ID", "")
    os.environ["_ENDPOINT"] = PROJECT_ENDPOINT
    os.environ["PROJECT_ENDPOINT"] = PROJECT_ENDPOINT
    
    print(f"✅ 설정 파일 '{config_file}'에서 환경 변수를 로드했습니다.")
    print(f"\n📌 Foundry Name: {FOUNDRY_NAME}")
    print(f"📌 Resource Group: {RESOURCE_GROUP}")
    print(f"📌 Location: {LOCATION}")
    print(f"📌 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")
    
except FileNotFoundError:
    print(f"⚠️ '{config_file}' 파일을 찾을 수 없습니다.")
    print("💡 01-setup.ipynb를 먼저 실행하여 환경을 설정하세요.")
    raise

# 필수 패키지 설치
%pip install -q azure-ai-projects azure-identity

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

print(f"\n💡 사용할 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")


## 리소스 모니터링

배포된 모델과 에이전트를 조회합니다.

In [ ]:
# Fleet Overview - 배포된 에이전트 상태 확인
credential = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

try:
    # 배포된 에이전트 목록 조회
    agents = list(client.agents.list())
    
    print("🚀 Fleet Overview - Running Agents")
    print("=" * 80)
    
    if not agents:
        print("\n⚠️ 배포된 에이전트가 없습니다.")
        print("💡 03-agents.ipynb를 먼저 실행하여 에이전트를 생성하세요.")
    else:
        print(f"\n📊 총 {len(agents)}개의 에이전트가 실행 중입니다:\n")
        
        for i, agent in enumerate(agents, 1):
            print(f"{i}. {agent.name}")
            print(f"   ID: {agent.id}")
            
            # Model 정보 (속성명이 다를 수 있음)
            model_name = getattr(agent, 'model', None) or getattr(agent, 'model_id', 'N/A')
            print(f"   Model: {model_name}")
            
            if hasattr(agent, 'tools') and agent.tools:
                tools_str = ", ".join([t.get('type', 'unknown') for t in agent.tools])
                print(f"   Tools: {tools_str}")
            
            if hasattr(agent, 'created_at'):
                print(f"   Created: {agent.created_at}")
            
            print(f"   Status: ✅ Active")
            print("-" * 80)
        
        print(f"\n✅ Fleet Overview 조회 완료!")
        
except Exception as e:
    print(f"⚠️ Fleet 정보 조회 실패: {e}")
    print("\n💡 해결 방법:")
    print("   1. Azure에 로그인했는지 확인 (az login)")
    print("   2. FOUNDRY_NAME이 올바른지 확인")
    print("   3. 프로젝트에 에이전트가 생성되었는지 확인")

## Quota 확인

모델별 할당량과 사용률을 확인합니다.

**⚠️ 참고**: 자세한 Quota 관리는 Azure Portal에서만 가능합니다.
- Operate > Quota에서 실시간 사용률 확인
- Quota 증가 요청

In [ ]:
# Assets 관리 - 모델, 연결, 도구 확인

print("📦 Assets 관리")
print("=" * 80)

try:
    # 1. 연결(Connections) 조회
    print("\n1️⃣ Connections (모델 연결):")
    print("-" * 80)
    
    connections = list(client.connections.list())
    
    if not connections:
        print("⚠️ 연결된 모델이 없습니다.")
        print("💡 Portal에서 모델을 배포하세요: https://ai.azure.com")
    else:
        for conn in connections:
            print(f"\n✅ {conn.name}")
            if hasattr(conn, 'connection_type'):
                print(f"   Type: {conn.connection_type}")
            if hasattr(conn, 'endpoint_url'):
                print(f"   Endpoint: {conn.endpoint_url}")
    
    # 2. 에이전트 목록 (Assets)
    print("\n\n2️⃣ Agents:")
    print("-" * 80)
    
    agents = list(client.agents.list())
    
    if agents:
        for agent in agents:
            print(f"\n✅ {agent.name}")
            print(f"   ID: {agent.id}")
            
            # Model 정보 (속성명이 다를 수 있음)
            model_name = getattr(agent, 'model', None) or getattr(agent, 'model_id', 'N/A')
            print(f"   Model: {model_name}")
    else:
        print("⚠️ 생성된 에이전트가 없습니다.")
    
    print("\n" + "=" * 80)
    print("✅ Assets 조회 완료!")
    
    # 3. 리소스 통계 시각화
    print("\n\n3️⃣ 리소스 통계 시각화:")
    print("-" * 80)
    
    # matplotlib 설치
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "matplotlib"])
    
    import matplotlib.pyplot as plt
    
    # 연결 및 에이전트 수 시각화
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # 연결 타입별 분포
    if connections:
        connection_types = {}
        for conn in connections:
            if hasattr(conn, 'connection_type'):
                conn_type = str(conn.connection_type)
                connection_types[conn_type] = connection_types.get(conn_type, 0) + 1
        
        if connection_types:
            ax1.bar(connection_types.keys(), connection_types.values(), color='skyblue')
            ax1.set_title('Connection Types Distribution', fontsize=14, fontweight='bold')
            ax1.set_xlabel('Connection Type')
            ax1.set_ylabel('Count')
            ax1.tick_params(axis='x', rotation=45)
    else:
        ax1.text(0.5, 0.5, 'No connections available', ha='center', va='center', fontsize=12)
        ax1.set_title('Connection Types Distribution', fontsize=14, fontweight='bold')
    
    # 에이전트 모델 분포
    if agents:
        agent_models = {}
        for agent in agents:
            model_name = getattr(agent, 'model', None) or getattr(agent, 'model_id', 'Unknown')
            agent_models[model_name] = agent_models.get(model_name, 0) + 1
        
        ax2.bar(agent_models.keys(), agent_models.values(), color='lightcoral')
        ax2.set_title('Agent Models Distribution', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Model')
        ax2.set_ylabel('Count')
        ax2.tick_params(axis='x', rotation=45)
    else:
        ax2.text(0.5, 0.5, 'No agents available', ha='center', va='center', fontsize=12)
        ax2.set_title('Agent Models Distribution', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\\n📊 리소스 통계 시각화 완료!")
    
except Exception as e:
    print(f"\n⚠️ Assets 조회 실패: {e}")
    print("💡 Portal에서 확인: https://ai.azure.com > Build > Assets")

## 리소스 정리 (선택사항)

워크샵 종료 후 비용 절감을 위해 리소스를 삭제할 수 있습니다.

In [ ]:
# Quota 관리 정보
print("📊 Quota 및 Rate Limiting 정보")
print("=" * 80)
print("\n💡 Quota 및 Rate Limiting은 주로 Azure Portal에서 관리됩니다.")
print("\n확인 방법:")
print("   1. Azure Portal: https://portal.azure.com")
print("   2. AI Services 리소스 선택")
print("   3. 'Quota' 또는 'Usage + quotas' 메뉴 확인")
print("\n주요 확인 항목:")
print("   - Token Per Minute (TPM): 분당 처리 가능한 토큰 수")
print("   - Requests Per Minute (RPM): 분당 처리 가능한 요청 수")
print("   - Provisioning Throughput Units (PTU): 프로비저닝된 처리 단위")
print("\n" + "=" * 80)
print("✅ Portal에서 실시간 quota 사용량을 확인하세요!")

## 📚 추가 리소스

- [Microsoft Foundry Control Plane](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/overview?view=foundry)
- [Agent 헬스 및 성능 모니터링](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/monitoring-across-fleet?view=foundry)
- [Custom Agent 등록](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/register-custom-agent?view=foundry)
- [Guardrail Policy 생성](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/quickstart-create-guardrail-policy?view=foundry)
- [규정 준수 및 보안 관리](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/how-to-manage-compliance-security?view=foundry)
- [모델 비용 및 성능 최적화](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/how-to-optimize-cost-performance?view=foundry)

## 마무리

### 축하합니다! 🎉

Microsoft Foundry Hands-on Workshop의 모든 모듈을 완료했습니다!

### 학습한 내용 요약

```
✅ Module 01: 환경 설정
   - Resource Group 및 Foundry 리소스 생성

✅ Module 02: 모델 및 배포
   - 모델 탐색, 배포, Model Router 구성

✅ Module 03: 에이전트 개발
   - 다양한 타입의 에이전트 생성 및 배포

✅ Module 04: Foundry IQ
   - AI Search 및 Blob Storage 기반 Knowledge Base 구축

✅ Module 05: 워크플로우
   - Sequential, Group Chat, Human-in-loop 워크플로우 구현

✅ Module 06: 평가
   - 에이전트 품질 평가 및 개선

✅ Module 07: Control Plane
   - 프로덕션 모니터링 및 관리
```

### 다음 단계

이제 다음을 수행할 준비가 되었습니다:

1. **프로덕션 배포**
   - 실제 애플리케이션에 Foundry 통합
   - CI/CD 파이프라인 구축
   - 모니터링 및 알림 설정

2. **고급 기능 탐색**
   - Custom tools 및 MCP servers 개발
   - Multi-agent 협업 패턴
   - Fine-tuning 및 모델 커스터마이징

3. **커뮤니티 참여**
   - [Microsoft Tech Community](https://techcommunity.microsoft.com) 포럼 참여
   - [GitHub Samples](https://github.com/Azure-Samples) 기여
   - 사용 사례 공유